In [1]:
ADDRESS = "0x99cd83938B953EA2C77944f5cF1c483D25CB71B0"

In [2]:
import os # File path manipulations
import json # Parsing the ABI and Metadata
from io import StringIO # For loading the image

import requests # For getting the metadata and image
import PIL # For encoding the image
from eth_account import Account # For managing private keys
from web3 import Web3 # For web3 interactions 
from IPython.display import display # For rich display

In [3]:
class CollectionItem:
    
    def __init__(self, id_ : int, contract, address: str):
        self._id = id_
        self._contract = contract
        self._address = address
        
    @property
    def uri(self) -> str:
        return self._contract.functions.uri(0).call().replace("{id}", format(self._id, 'x').zfill(64))
    
    @property
    def balance(self) -> int:
        return self._contract.functions.balanceOf(self._address, self._id).call()
    
    @property
    def supply(self) -> int:
        return self._contract.functions.totalSupply(self._id).call()
    
    @property
    def metadata(self) -> dict:
        return requests.get(self.uri).json()
    
    @property
    def image(self):
        r = requests.get(self.metadata["image"])
        return PIL.Image.open(StringIO(r.content))
    
    def _repr_html_(self) -> str:
        meta = self.metadata
        return '<h3>{}</h3><figure><img src="{}" width=250 height=250 /><figcaption>{} <br> Balance: {} of {}</figcaption></figure>'.format(
            meta["name"], meta["image"], meta["description"], self.balance, self.supply)

In [4]:
def init_w3(rpc_url: str):
    return Web3(Web3.HTTPProvider(rpc_url))

def init_contract(w3, address: str):
    abi_path = os.path.join("./", "contracts", "PoktWorkshop.json")

    with open(abi_path, "r") as f:
        abi = json.load(f)
    return w3.eth.contract(address=address, abi=abi["abi"])


def init_account(private_key):
    return Account.from_key(private_key)

In [5]:
RPC_URL = "Portal Endpoint URL" # Replace with your Pocket Endpoint
PRIVATE_KEY = "Private Key Here" # Replace with your own key

In [6]:
w3 = init_w3(RPC_URL)
contract = init_contract(w3, ADDRESS)
account = init_account(PRIVATE_KEY)

In [7]:
def claim_item(w3, contract, account, **txParams):
    claim_tx = contract.functions.claim().buildTransaction({'from': account.address, 'nonce' : w3.eth.get_transaction_count(account.address)})
    signed_tx = account.signTransaction(claim_tx)
    w3.eth.send_raw_transaction(signed_tx.rawTransaction)

def total_items(contract):
    return contract.functions.lastClaimableId().call()

def view_owned_items(contract, address):
    n = total_items(contract)
    items = [CollectionItem(i, contract, address) for i in range(1, n+1)]
    return [item for item in items if item.balance]

def set_next_item(w3, contract, account):
    next_tx = contract.functions.setNextItem(28800).buildTransaction({'from': account.address, 'nonce' : w3.eth.get_transaction_count(account.address)})
    signed_tx = account.signTransaction(next_tx)
    w3.eth.send_raw_transaction(signed_tx.rawTransaction)

In [8]:
items = view_owned_items(contract, account.address)
for item in items:
    display(item)
if not items:
    print("Not Items Currently Owned")

Not Items Currently Owned


In [9]:
claim_item(w3, contract, account)

In [12]:
items = view_owned_items(contract, account.address)
for item in items:
    display(item)